# Imports

In [ ]:
from paraview.simple import *
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cuml
from ipywidgets import fixed, interact, interactive, VBox, HBox
from ipyparaview.widgets import PVDisplay
import ipywidgets as widgets
import PVGeo

## Load and visualize MNIST digits (via Scikit-learn)

In [ ]:
X, Y = load_digits(return_X_y=True)

In [ ]:
def plotDigit(linear_img, N=8, label=None):
    img2d = linear_img.reshape((N,N))
    plt.imshow(img2d, cmap='binary')
    if label is not None:
        plt.title("Digit: '{}'".format(str(label)))
    plt.show()
    
k=10
plotDigit(X[k, :], label=Y[k])

## Cluster data with UMAP

In [ ]:
%%time
umap = cuml.manifold.UMAP(n_components=3)
X3 = umap.fit_transform(X, Y)

In [ ]:
Xdf = pd.DataFrame(X3, columns=["X", "Y", "Z"])
Xdf["color"] = Y

## Setup paraview objects

In [ ]:
renderView1 = CreateView('RenderView')
renderView1.AxesGrid.Visibility = 1
renderView1.AxesGrid.ShowGrid = 1
renderView1.ViewSize = [640, 480]
M3p = TrivialProducer()

M3p.GetClientSideObject().SetOutput(PVGeo.interface.data_frame_to_table(Xdf))
M3p.UpdatePipeline()
tableToPoints1 = TableToPoints(Input=M3p)
tableToPoints1.XColumn = 'X'
tableToPoints1.YColumn = 'Y'
tableToPoints1.ZColumn = 'Z'
colorLUT = GetColorTransferFunction('color')
colorLUT.ApplyPreset('jet', True)
colorLUT.RescaleTransferFunction(0.0, 9.0)
tableToPoints1Display = Show(tableToPoints1, renderView1)
tableToPoints1Display.RenderPointsAsSpheres = 1
tableToPoints1Display.Representation = 'Surface'
tableToPoints1Display.ColorArrayName = ['POINTS', 'color']
tableToPoints1Display.LookupTable = colorLUT
tableToPoints1Display.RenderPointsAsSpheres = 1
tableToPoints1Display.PointSize = 6.0

def update_data(new_value):
    data = new_value
    M3p.GetClientSideObject().SetOutput(PVGeo.interface.data_frame_to_table(new_value))
    M3p.UpdatePipeline()
    tableToPoints1.Input = M3p


In [ ]:
pv_widget = PVDisplay(renderView1)

## Prepare PV widget with interactive controls

In [ ]:
def rayTracing(use_rt):
    if use_rt:
        renderView1.EnableRayTracing = 1
        renderView1.BackEnd = 'OptiX pathtracer'
        renderView1.SamplesPerPixel = 7
    else:
        renderView1.EnableRayTracing = 0
        
def minDist(min_dist):
    umap = cuml.manifold.UMAP(n_components=3, min_dist=min_dist)
    X3 = umap.fit_transform(X, Y)
    Xdf = pd.DataFrame(X3, columns=["X", "Y", "Z"])
    Xdf["color"] = Y
    update_data(Xdf)

In [ ]:
VBox([pv_widget,
      HBox([interactive(rayTracing, use_rt=widgets.Checkbox(value=False)),
            interactive(minDist, min_dist=widgets.FloatSlider(value=0.1, min=0.01, max=2, 
                                                              continuous_update=False))])])
      